In [ ]:
import matplotlib
import argparse
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch
import torch.nn as nn # import modules

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid
from utils.options import args_parser
from models_v2.Update import LocalUpdate
from models_v2.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist2, MobileNetV2, LeNet5, LeNet10
from models_v2.Fed import FedAvg
from models_v2.Fed import FedQAvg, Quantization, Quantization_Finite, my_score, my_score_Finite
from models_v2.test import test_img

%load_ext autoreload
%autoreload 2
from torch.autograd import grad
parser = argparse.ArgumentParser(description='Deep Leakage from Gradients.')
parser.add_argument('--index', type=int, default="25",
                    help='the index for leaking images on CIFAR.')
parser.add_argument('--image', type=str,default="",
                    help='the path to customized image.')
args = parser.parse_args([])
from torch.autograd import grad

In [ ]:
class my_argument:    
    epochs = 400    #"rounds of training"
    num_users = 5  # "number of users: K"
    frac = 0.5 #"the fraction of clients: C"
    local_ep=5 #"the number of local epochs: E"
    local_bs=10000 #"local batch size: B"
    bs=128 #"test batch size"
    lr=0.001 #"learning rate"
    momentum=0.5 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"
    weight_decay = 5e-4
    opt = 'ADAM'

    # model arguments
    model = 'cnn'
    kernel_num=9 #, help='number of each kind of kernel')
    kernel_sizes='3,4,5' #  help='comma-separated kernel size to use for convolution')
    norm='batch_norm' #, help="batch_norm, layer_norm, or None")
    num_filters=32 #, help="number of filters for conv nets")
    max_pool='True' #help="Whether use max pooling rather than strided convolutions")

    # other arguments
    dataset='cifar' #, help="name of dataset")
    iid=1
    num_classes=10#, help="number of classes")
    num_channels=3#, help="number of channels of imges")
    gpu=1#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    verbose='False'#, help='verbose print')
    seed=1#, help='random seed (default: 1)')
    
args = my_argument()

args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')


use_cuda = torch.cuda.is_available()
print(use_cuda)
args.device = torch.device("cpu")
args.device = torch.device("cuda" if use_cuda else "cpu")
print(args.device)

In [ ]:
from models_v2.Fed import FedAdd,FedSubstract,weight_vectorization_gen,FedAvg_gradient, weight_vectorization_gen2
net_glob = LeNet10().to(args.device)  #(args=args).to(args.device)
w_glob=net_glob.state_dict()
g, dim = weight_vectorization_gen(w_glob)
#print(len(g))
#print(net_glob)

In [ ]:
net_glob=torch.hub.load('pytorch/vision:v0.10.0','resnet18',pretrained=False)
net_glob.eval()
w_glob = net_glob.state_dict()
g, dim = weight_vectorization_gen(w_glob)
print(len(g))
#print(net_glob)

In [ ]:
from collections import defaultdict

arch = 'ConvNet64'
arch='cnn'
arch='ResNet18'
num_images = args.num_users
trained_model = False

In [ ]:
def toNumpyFlatArray(self):
        return self.flat


def updateFromNumpyFlatArray(self, arr):
    self.flat = arr
    start = 0
    new_glob = OrderedDict()
    for k in self.w_glob.keys():
        size = 1
        for dim in self.w_glob[k].shape:
            size *= dim
        shaped = np.reshape(arr[start : start + size].copy(), self.w_glob[k].shape)
        new_glob[k] = torch.from_numpy(shaped)
        start = start + size
    self.w_glob = new_glob
    self.net_glob.load_state_dict(self.w_glob)

In [ ]:
import inversefed
setup = inversefed.utils.system_startup()
defs = inversefed.training_strategy('conservative')

loss_fn, trainloader, validloader =  inversefed.construct_dataloaders('CIFAR10', defs)

net_glob, _ = inversefed.construct_model(arch, num_classes=10, num_channels=3)
net_glob.to(**setup)
# net_glob=torch.hub.load('pytorch/vision:v0.10.0','resnet18',pretrained=False)
# net_glob.eval()
if trained_model:
    epochs = 10
    file = f'{arch}_{epochs}.pth'
    try:
        net_glob.load_state_dict(torch.load(f'models_v2/{file}'))
    except FileNotFoundError:
        inversefed.train(net_glob, loss_fn, trainloader, validloader, defs, setup=setup)
        torch.save(net_glob.state_dict(), f'models_v2/{file}')
net_glob.eval();
w_glob=net_glob.state_dict()
g, dim = weight_vectorization_gen(w_glob)
print(len(g))

In [ ]:
dm = torch.as_tensor(inversefed.consts.cifar10_mean, **setup)[:, None, None]
ds = torch.as_tensor(inversefed.consts.cifar10_std, **setup)[:, None, None]
def plot(tensor):
    tensor = tensor.clone().detach()
    tensor.mul_(ds).add_(dm).clamp_(0, 1)
    if tensor.shape[0] == 1:
        return plt.imshow(tensor[0].permute(1, 2, 0).cpu());
    else:
        fig, axes = plt.subplots(1, tensor.shape[0], figsize=(12, tensor.shape[0]*12))
        for i, im in enumerate(tensor):
            axes[i].imshow(im.permute(1, 2, 0).cpu());





In [ ]:
# op=torch.load("frog_blue_topK_new_1000.pt")
# plot(op)

In [ ]:
num_users=args.num_users
ground_truth, labels = [], []
ground_truth_joint,labels_joint=[],[]
for i in range(num_users):
    ground_truth.append([])
    labels.append([])
idx = 3010 # choosen randomly ... just whatever you want
    #while len(labels) < num_images:
index=[1290,1245,1110,410,1270]
#index=[301,302,303,304,305]
#index=[]
#index=[2661, 5723, 6633, 7758, 1245 ]
#index=[2661, 5723, 303, 7758, 1245 ]
#index=[1290, 1245, 303, 410, 5585]
#1230
#1269
#801
#1274
#1280
#1283
#1288
idx=0
index=[5723, 410, 2661, 1245, 5585 ]
index= [305, 410, 4434, 1455, 8034]
#index=[301, 302, 303, 304, 305]
index=[303, 305, 4434, 410, 5723]
index=[4434, 5723, 410, 1455,467 ]
index= [1567, 1896, 4434, 3149, 770]
index = [1567, 3149, 3532, 626, 1455]
index = [1567, 3149, 3532, 1694, 770]
index=[4434, 3149, 3532, 410,1245]
index= [4434, 3149, 3532, 410, 1455]
index=[7049, 3149, 3532,2889, 1245]
index= [4434, 3149, 3532, 3621, 1245]
for i in range(num_users):
    print(i)
    img, label = validloader.dataset[index[idx]]
    idx += 1
    labels[i].append(torch.as_tensor((label,), device=setup['device']))
    ground_truth[i].append(img.to(**setup))
    ground_truth[i] = torch.stack(ground_truth[i])
    ground_truth_joint.append(img.to(**setup))
    
    labels[i] = torch.cat(labels[i])
    labels_joint.append(torch.as_tensor((label,), device=setup['device']))
    #plot(ground_truth[i]);
    #plt.show()
    print([validloader.dataset.classes[l] for l in labels[i]]);


# In[35]:


ground_truth_joint = torch.stack(ground_truth_joint)
labels_joint = torch.cat(labels_joint)


# In[36]:


plot(ground_truth_joint);
plt.savefig("original2.png")
print([validloader.dataset.classes[l] for l in labels_joint]);


In [ ]:
plot(ground_truth[2])
print(labels[2])

In [ ]:
# a=np.zeros((10000000,300,25),dtype=np.uint16)
# np.save('arr.npy',a)

# 1. FedAvg with A=0

In [ ]:

local_lr = 1e-4
local_steps = 1
use_updates = True

In [ ]:
# A_conc1=np.load("A_conc1_in.npy")
# A_conc2=np.load("A_conc2_in.npy")
# A_conc3=np.load("A_conc3_in.npy")
# A_conc1=np.load("A_conc4_in.npy")
# A_conc2=np.load("A_conc5_in.npy")
# A_conc3=np.load("A_conc6_in.npy")
# err1=np.load("err1_in.npy")
# err2=np.load("err2_in.npy")
# err3=np.load("err3_in.npy")
# err1=np.load("err4_in.npy")
# err2=np.load("err5_in.npy")
# err3=np.load("err6_in.npy")
# T=np.load("T.npy")

In [ ]:
# We partition the entire gradient into 6 segments and create A matrix and gradient sum for each segment separately 
#to allow parallel computation

In [ ]:

from models_v2.Fed import FedAdd,FedSubstract,weight_vectorization_cifar,FedAvg_gradient, weight_vectorization_gen, weight_vectorization_gen2
#net_glob = LeNet10().to(args.device)
#net_glob.train()
import torchvision.models as models
# dev=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# net_glob =models.resnet18(pretrained=False, progress=True)
# net_glob=net_glob.to(dev)
# copy weights
#net_glob=torch.hub.load('pytorch/vision:v0.10.0','resnet18',pretrained=False)
#net_glob.eval()
w_glob=net_glob.state_dict()
g, dim = weight_vectorization_gen(w_glob)
print(len(g))
w_glob = net_glob.state_dict()
w_glob=net_glob.state_dict()
g, dim = weight_vectorization_gen(w_glob)
print(len(g))
# training
loss_train = []
loss_test_arr = []
acc_test_arr = []
cv_loss, cv_acc = [], []
val_loss_pre, counter = 0, 0
net_best = None
best_loss = None
val_acc_list, net_list = [], []

A=torch.load("Model.pt")
net_glob.load_state_dict(A)
w_glob = net_glob.state_dict()
m_local=[]
d=11699132 #resnet18
d= 11173962 
#d=2904970 #convnet64

K_local=round(0.01*d)
#K_local=round(1*d)

iter_no=500
avg=[]
error=[]

    #print(len(idxs_users))
user_no=args.num_users
idxs_users=range(0,args.num_users)
updated=[]
model_diff=[]
grad_vect=[]
prev=[]
original_gradient=[] 
accumu_grad=[]
grad_vect_send=[]
store_grad=[]
sparsity=0.5

#abs_vect,layer_size=weight_vectorization_gen(w_glob) # vectorizing the global gradient
#print(abs_vect)
    #finding top- K_global locations in global gradient

A=[]
e=[]
#A=np.zeros(iter_no)

#for iter in range(iter_no):
    
    #A[iter]=np.zeros(args.num_users)
for i in range(len(idxs_users)):
    error.append(np.zeros((d,1))[:,0])
    #m_local.append(np.zeros((d,1),dtype=np.uint16))
#updated=[None]*args.num_users
#model_diff=[None]*args.num_users
grad_vect=[None]*args.num_users
#prev=[None]*args.num_users
    
grad_vect_send=[None]*args.num_users
original_gradient=[None]*args.num_users
#accumu_grad=[None]*args.num_users
    
    #store_grad.append([])
net_glob.zero_grad()
input_gradient=[]
for i in range(args.num_users):
    input_gradient.append([])   
    
    
target=range(0,d)

user_no=args.num_users


#ori_grad_element=[]


#A_big=[None]*target[0]
#for i in target:
e=[[]]*len(target)
#e=[[]]*target[0]
T=[]
#A_big=np.zeros((len(target),iter_no,user_no))
# e=np.empty((len(target),iter_no))
# T=np.empty((len(target),user_no))
T=[]
for i in range(user_no):
    T.append(np.zeros(11173962,dtype=np.uint8))
A_big=[]
A_big2=[]
A_big3=[]
A_big4=[]
A_big5=[]
A_big6=[]
for i in range(iter_no):
    A_big.append([])
    A_big2.append([])
    A_big3.append([])
    A_big4.append([])
    A_big5.append([])
    A_big6.append([])
    for j in range(user_no):
        A_big[i].append([])
        A_big2[i].append([])
        A_big3[i].append([])
        A_big4[i].append([])
        A_big5[i].append([])
        A_big6[i].append([])
ind1=int(d/6)
ind2=int(2*d/6)
ind3=int(3*d/6)
ind4=int(4*d/6)
ind5=int(5*d/6)
ind6=int(6*d/6)
# A_big=np.empty((iter_no,user_no))
# A_big2=np.empty((iter_no,user_no))
# A_big3=np.empty((iter_no,user_no))
f=[]
A_conc1=[]
A_conc2=[]
A_conc3=[]
A_conc4=[]
A_conc5=[]
A_conc6=[]
err1=[]
err2=[]
err3=[]
err4=[]
err5=[]
err6=[]
e=[]
#T=[]
# for i in range(user_no):
#     T.append(np.random.choice([0, 1], size=11173962, p=[.5, .5]))
    
net_glob.zero_grad()   
for iter in range(iter_no): #args.epochs
    print("iteration no.",iter)
    m_local=[]
    m_local=np.zeros(d,dtype=np.uint8)
    grad_vect_send=[]
    e.append(np.zeros(d))
    f=[]
        #T=[]
    w_locals, loss_locals,diff_locals,grad_locals = [], [],[],[]
    m = 10
    #idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    
        #T=np.zeros(user_no)
    g=[]
    s=[]
    for user in idxs_users:
        #print(user)
        m_local=np.zeros(d,dtype=np.uint8)
            
        #m_local.append(np.zeros(d,dtype=np.uint8))
        #grad_vect.append(np.zeros(d,dtype=np.uint16))
        #updated[user]=copy.deepcopy(w_glob)
        #local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[user])
        #w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
        target_loss, _, _ = loss_fn(net_glob(ground_truth[user]), labels[user])
#target_loss, _, _ = loss_fn(net(ground_truth), labels)
    #input_gradient[i] = torch.autograd.grad(target_loss, model.parameters())
        input_gradient[user]=inversefed.reconstruction_algorithms.loss_steps(net_glob, ground_truth[user], labels[user], 
                                                        lr=local_lr, local_steps=local_steps,
                                                                   use_updates=use_updates)
        
        
    #print(input_gradient[i])
        input_gradient[user] = [grad.detach() for grad in input_gradient[user]]
        g, dim = weight_vectorization_gen2(input_gradient[user])
        #print(len(g))
        
        #prev[user]=updated[user]
 
        
        if iter==0:
            original_gradient[user]=g
            #print("original gradient")
            #print(original_gradient[user])
        #print(layer_size)
        g=g[:,0]+error[user] # error feedback
        #ori_grad_element[user]=original_gradient[user][target]
        
        grad1=abs(np.array(g))
        location_local=np.argpartition(grad1,-K_local)[-K_local:]
        
        #location_local= np.random.choice(range(d),K_local,replace=False)
        # storing top- K_local locations for each user
        a=np.sort(location_local)
        print("location of user", user)
        print(a)
        #print(a[0:10])
        
        for i in location_local:
            m_local[i]=np.uint8(1)# creating local mask
        #mask=m_local[user]
        #print(m_local[user][ind1:ind2])
        grad_vect_send=np.multiply(m_local,g)
        error[user]=g-grad_vect_send
        f=np.uint8(1)*m_local
        count=0
        if iter==0:
            T[user]=-1*(f)
      
        if user==0:
            e[iter]=f*(iter-T[user])*original_gradient[user].reshape(d)
        else:
            e[iter]+=f*(iter-T[user])*original_gradient[user].reshape(d)
        print(iter-T[user])
        A_big[iter][user]=np.uint16(f[0:ind1]*(iter-T[user][0:ind1]))
        A_big2[iter][user]=np.uint16(f[ind1:ind2]*(iter-T[user][ind1:ind2]))
        A_big3[iter][user]=np.uint16(f[ind2:ind3]*(iter-T[user][ind2:ind3]))
        A_big4[iter][user]=np.uint16(f[ind3:ind4]*(iter-T[user][ind3:ind4]))
        A_big5[iter][user]=np.uint16(f[ind4:ind5]*(iter-T[user][ind4:ind5]))
        A_big6[iter][user]=np.uint16(f[ind5:ind6]*(iter-T[user][ind5:ind6]))
            #for user in idxs_users:
        T[user]=T[user]+f*(-T[user]+iter)
    A_conc1.append(np.array(A_big[iter]).transpose())
    A_conc2.append(np.array(A_big2[iter]).transpose())
    A_conc3.append(np.array(A_big3[iter]).transpose())
    A_conc4.append(np.array(A_big4[iter]).transpose())
    A_conc5.append(np.array(A_big5[iter]).transpose())
    A_conc6.append(np.array(A_big6[iter]).transpose())
    err1.append(np.array(e[iter][0:ind1]).transpose())
    err2.append(np.array(e[iter][ind1:ind2]).transpose())
    err3.append(np.array(e[iter][ind2:ind3]).transpose())
    err4.append(np.array(e[iter][ind3:ind4]).transpose())
    err5.append(np.array(e[iter][ind4:ind5]).transpose())
    err6.append(np.array(e[iter][ind5:ind6]).transpose())
    del g
    del grad1
    del m_local
    #del mask
    del location_local
    del a
    del f
    
del A_big
del A_big2
del A_big3
del A_big4
del A_big5
del A_big6
del e
#del T
torch.cuda.empty_cache()       
               #print("T[t][user]",user)
               #print(T[t][user])
       
#     grad_avg=FedAvg_gradient(grad_locals) # taking the average of masked gradients


In [ ]:
from scipy.sparse import csr_matrix, save_npz, load_npz
#print(A_conc1.shape)
#A_co=np.zeros((1,))
A_con=[]
for i in range(iter_no):
    A_con.append(csr_matrix(np.array(A_conc1[i])))
    #A_con[i]=A_con[i].todense()
#print(A_con.todense())
#A_con=np.array(A_con)
A_conc11=A_con
np.save("A_conc1_in_topK.npy", A_conc11)
del A_conc11
print("1")

A_con=[]
for i in range(iter_no):
    A_con.append(csr_matrix(np.array(A_conc2[i])))
    #A_con[i]=A_con[i].todense()
#print(A_con.todense())
#A_con=np.array(A_con)
A_conc22=A_con
np.save("A_conc2_in_topK.npy", A_conc22)
del A_conc22
print("2")

A_con=[]
for i in range(iter_no):
    A_con.append(csr_matrix(np.array(A_conc3[i])))
    #A_con[i]=A_con[i].todense()
#print(A_con.todense())
#A_con=np.array(A_con)
A_conc33=A_con
np.save("A_conc3_in_topK.npy", A_conc33)
del A_conc33
print("3")

A_con=[]
for i in range(iter_no):
    A_con.append(csr_matrix(np.array(A_conc4[i])))
    #A_con[i]=A_con[i].todense()
#print(A_con.todense())
A_con=np.array(A_con)
A_conc44=A_con
np.save("A_conc4_in_topK.npy", A_conc44)
del A_conc44
print("4")

A_con=[]
for i in range(iter_no):
    A_con.append(csr_matrix(np.array(A_conc5[i])))
    #A_con[i]=A_con[i].todense()
#print(A_con.todense())
#A_con=np.array(A_con)
A_conc55=A_con
np.save("A_conc5_in_topK.npy", A_conc55)
del A_conc55
print("5")

A_con=[]
for i in range(iter_no):
    A_con.append(csr_matrix(np.array(A_conc6[i])))
    #A_con[i]=A_con[i].todense()
#print(A_con.todense())
#A_con=np.array(A_con)
A_conc66=A_con
np.save("A_conc6_in_topK.npy", A_conc66)
del A_conc66
print("6")
# print(A_con)
# print(np.array(A_conc1).nbytes)
# print(A_con.nbytes)

In [ ]:
# from scipy.sparse import csr_matrix, save_npz, load_npz
# #print(A_conc1.shape)
# #A_co=np.zeros((1,))
# A_con=[]
# for i in range(iter_no):
#     A_con.append(csr_matrix(np.array(A_conc1[i])))
#     #A_con[i]=A_con[i].todense()
# #print(A_con.todense())
# #A_con=np.array(A_con)
# A_conc11=A_con
# np.save("A_conc1_in.npy", A_conc11)
# del A_conc11
# print("1")

# A_con=[]
# for i in range(iter_no):
#     A_con.append(csr_matrix(np.array(A_conc2[i])))
#     #A_con[i]=A_con[i].todense()
# #print(A_con.todense())
# #A_con=np.array(A_con)
# A_conc22=A_con
# np.save("A_conc2_in.npy", A_conc22)
# del A_conc22
# print("2")

# A_con=[]
# for i in range(iter_no):
#     A_con.append(csr_matrix(np.array(A_conc3[i])))
#     #A_con[i]=A_con[i].todense()
# #print(A_con.todense())
# #A_con=np.array(A_con)
# A_conc33=A_con
# np.save("A_conc3_in.npy", A_conc33)
# del A_conc33
# print("3")

# A_con=[]
# for i in range(iter_no):
#     A_con.append(csr_matrix(np.array(A_conc4[i])))
#     #A_con[i]=A_con[i].todense()
# #print(A_con.todense())
# A_con=np.array(A_con)
# A_conc44=A_con
# np.save("A_conc4_in.npy", A_conc44)
# del A_conc44
# print("4")

# A_con=[]
# for i in range(iter_no):
#     A_con.append(csr_matrix(np.array(A_conc5[i])))
#     #A_con[i]=A_con[i].todense()
# #print(A_con.todense())
# #A_con=np.array(A_con)
# A_conc55=A_con
# np.save("A_conc5_in.npy", A_conc55)
# del A_conc55
# print("5")

# A_con=[]
# for i in range(iter_no):
#     A_con.append(csr_matrix(np.array(A_conc6[i])))
#     #A_con[i]=A_con[i].todense()
# #print(A_con.todense())
# #A_con=np.array(A_con)
# A_conc66=A_con
# np.save("A_conc6_in.npy", A_conc66)
# del A_conc66
# print("6")
# # print(A_con)
# # print(np.array(A_conc1).nbytes)
# # print(A_con.nbytes)

In [ ]:
# np.save("err1_in.npy", err1)
# np.save("err2_in.npy", err2)
# np.save("err3_in.npy", err3)
# np.save("err4_in.npy", err4)
# np.save("err5_in.npy", err5)
# np.save("err6_in.npy", err6)
# print("finish")
# np.save("T_randK.npy", T)
# print("finish")
# np.save("original_grad_randK.npy",original_gradient)

In [ ]:
np.save("err1_in_topK.npy", err1)
print("finish")
np.save("err2_in_topK.npy", err2)
print("finish")
np.save("err3_in_topK.npy", err3)
print("finish")
np.save("err4_in_topK.npy", err4)
print("finish")
np.save("err5_in_topK.npy", err5)
print("finish")
np.save("err6_in_topK.npy", err6)
print("finish")
np.save("T.npy", T)
print("finish")
np.save("original_grad.npy",original_gradient)
print("finish")

In [ ]:
# np.save("A_conc1_in.npy", A_conc11)
# #save_npz("A_conc1_in.npz", A_conc1)
# np.save("A_conc2_in.npy", A_conc22)
# #save_npz("A_conc2_in.npz", A_conc2[0])
# np.save("A_conc3_in.npy", A_conc33)
# #save_npz("A_conc3_in.npz", A_conc3[4])
# np.save("A_conc4_in.npy", A_conc44)
# #save_npz("A_conc4_in.npz", A_conc4)
# np.save("A_conc5_in.npy", A_conc55)
# #save_npz("A_conc5_in.npz", A_conc5)
# np.save("A_conc6_in.npy", A_conc66)
#save_npz("A_conc6_in.npz", A_conc6)
# np.save("err1_in.npy", err1)
# np.save("err2_in.npy", err2)
# np.save("err3_in.npy", err3)
# np.save("err4_in.npy", err4)
# np.save("err5_in.npy", err5)
# np.save("err6_in.npy", err6)
# np.save("T.npy", T)
# np.save("original_grad.npy",original_gradient)

In [ ]:
del A_conc11
del A_conc22
del A_conc33
del A_conc44
del A_conc55
del A_conc66
torch.cuda.empty_cache()   

In [ ]:
print("finish")